In [1]:
%load_ext autoreload
%autoreload 2
PROJ_HOME='/Users/keemsunguk/Projects/analyst_performance/'
import sys
sys.path.append(PROJ_HOME)
import gc
from datetime import datetime

In [2]:
import pandas as pd
import copy
import pickle
from tqdm import tqdm

In [3]:
from pathlib import Path

In [4]:
data_dir = Path(PROJ_HOME+'/data')

In [5]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
from googletrans import Translator

In [8]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])

In [9]:
ex1 = translator.translate('하반기부터 반영될 가격상승과 가동률 개선 효과', src="ko", dest="en")
ex1.text

'Improvement of prices to be reflected from 2H'

# Loading translated data

In [10]:
#data_fnm = '/data/merged_translated_new.pkl'
data_fnm = '/data/pub_traded_only.pkl'
merged_df = pd.read_pickle(PROJ_HOME+data_fnm)

In [11]:
merged_df

,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,closing,...,고가44,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title
0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 실적은 기대수준을 소폭 하회할 전망,51300,50000,...,41300.0,41000.0,39400.0,40450.0,40450.0,40550.0,36400.0,51400,51400,4Q09 earnings are expected to meet expectations
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,폴리실리콘 원가개선 필요한 시점,62700,62800,...,52200.0,52000.0,50100.0,48450.0,48200.0,46300.0,39150.0,65900,67300,Polysilicon cost improvement required
2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,"견조한 고무사업, 그러나 부진한 페놀유도체",77300,76000,...,66100.0,65900.0,64200.0,62800.0,63200.0,63700.0,56300.0,82100,82100,"A solid rubber business, but sluggish phenol oil"
3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,20년 연초에 꾸는 꿈,237000,232000,...,324500.0,323500.0,305500.0,298000.0,301500.0,291500.0,284500.0,322500,351000,Dreams of 20 years
4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 ESS 비용반영을 긍정적으로 보자,321000,314000,...,412000.0,404000.0,385500.0,380000.0,377500.0,367000.0,343500.0,391500,422500,Let's see a positive reflection of ESS costs i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,182500,188000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"188,000","188,000",Room and comfortable 2022
39796,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,10850,10950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,950","10,950",Welcome to the confusion of the confidence
39797,심은주,하나금융,A271560,오리온,9/10/21,9/9/21,9/8/21,저가 매수 유효,124500,123500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"124,500","124,500",Low cost volume valid
39798,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지",8840,8770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"8,880","8,880","Hidden pearls, 3Q Human & Renewable Energy"


In [12]:
merged_df['recommendation'].value_counts()

BUY              23211
매수                7057
NR                6258
HOLD              1556
중립                 388
MarketPerform      263
Trading Buy        158
Neutral             99
Outperform          96
STRONG BUY          67
없음                  59
보유                  27
Underperform        11
비중축소                 8
Sell                 5
REDUCE               4
Name: recommendation, dtype: int64

In [13]:
title_sentiment = []
nlp.add_pipe('spacytextblob')
for k, r in tqdm(merged_df.iterrows(), total=39800):
    try:
        doc = nlp(r['en_report_title'])
        title_sentiment.append(doc._)
    except:
        print(k, r['en_report_title'])
        title_sentiment.append(r['en_report_title'])

 99%|█████████▊| 39267/39800 [03:00<00:02, 217.05it/s]


In [15]:
title_sentiment[4].assessments

[(['positive'], 0.22727272727272727, 0.5454545454545454, None)]

In [16]:
title_sentiment[0].assessments

[(['expected'], -0.1, 0.4, None)]

In [17]:
doc = nlp('A solid rubber business,')

In [18]:
doc._.assessments

[(['solid'], 0.0, 0.1, None)]

In [19]:
doc._.polarity

0.0

In [20]:
ex1 = translator.translate('견조한고무사업, 그러나 부진한페놀유도체', src="ko", dest="en")
ex1.text

'A solid rubber business, but sluggish aphon oxide'

In [21]:
doc = nlp(ex1.text)
doc._.assessments

[(['solid'], 0.0, 0.1, None)]

In [22]:
rt_sent = [ts.polarity for ts in title_sentiment]

In [23]:
merged_df['sentiment'] = rt_sent

In [24]:
merged_df[['report_title', 'en_report_title', 'sentiment']].head(10)

,report_title,en_report_title,sentiment
0,4분기 실적은 기대수준을 소폭 하회할 전망,4Q09 earnings are expected to meet expectations,-0.100000
1,폴리실리콘 원가개선 필요한 시점,Polysilicon cost improvement required,0.000000
2,"견조한 고무사업, 그러나 부진한 페놀유도체","A solid rubber business, but sluggish phenol oil",0.000000
3,20년 연초에 꾸는 꿈,Dreams of 20 years,0.000000
4,4분기 ESS 비용반영을 긍정적으로 보자,Let's see a positive reflection of ESS costs i...,0.227273
5,SK바이오팜의 공모가와 특별배당의 규모에 관심 필요,SK Bio Palm's contests and special dividends n...,0.321429
6,마케팅을 적게 해도 수요는 작아지지 않는다,"Even if you have less marketing, demand is not...",-0.020833
7,배당금 증액 여부에 관심 필요,I need to be interested in dividend increase,0.250000
8,4분기 실적은 시장예상치에 부합할 전망,4Q earnings are expected to meet market estimates,-0.100000
9,"업황 개선에도, 1Q20까지는 보수적 전망 불가피","In the improvement of the market,",0.000000


In [25]:
merged_df.at[22001, 'report_title']

'백화점과 면세점 모두 좋아지고 있다'

In [26]:
merged_df

,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,closing,...,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title,sentiment
0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 실적은 기대수준을 소폭 하회할 전망,51300,50000,...,41000.0,39400.0,40450.0,40450.0,40550.0,36400.0,51400,51400,4Q09 earnings are expected to meet expectations,-0.100000
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,폴리실리콘 원가개선 필요한 시점,62700,62800,...,52000.0,50100.0,48450.0,48200.0,46300.0,39150.0,65900,67300,Polysilicon cost improvement required,0.000000
2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,"견조한 고무사업, 그러나 부진한 페놀유도체",77300,76000,...,65900.0,64200.0,62800.0,63200.0,63700.0,56300.0,82100,82100,"A solid rubber business, but sluggish phenol oil",0.000000
3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,20년 연초에 꾸는 꿈,237000,232000,...,323500.0,305500.0,298000.0,301500.0,291500.0,284500.0,322500,351000,Dreams of 20 years,0.000000
4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 ESS 비용반영을 긍정적으로 보자,321000,314000,...,404000.0,385500.0,380000.0,377500.0,367000.0,343500.0,391500,422500,Let's see a positive reflection of ESS costs i...,0.227273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,182500,188000,...,0.0,0.0,0.0,0.0,0.0,0.0,"188,000","188,000",Room and comfortable 2022,0.400000
39796,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,10850,10950,...,0.0,0.0,0.0,0.0,0.0,0.0,"10,950","10,950",Welcome to the confusion of the confidence,0.800000
39797,심은주,하나금융,A271560,오리온,9/10/21,9/9/21,9/8/21,저가 매수 유효,124500,123500,...,0.0,0.0,0.0,0.0,0.0,0.0,"124,500","124,500",Low cost volume valid,0.000000
39798,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지",8840,8770,...,0.0,0.0,0.0,0.0,0.0,0.0,"8,880","8,880","Hidden pearls, 3Q Human & Renewable Energy",-0.083333


In [27]:
len(merged_df.columns.to_list())

68

In [28]:
merged_df.to_pickle(PROJ_HOME+'/data/sentiment_pub_trade.pkl')   #merged_sentiment_new.pkl')

In [29]:
dict(merged_df.iloc[0])

{'author': '백영찬',
 'source': 'KB',
 'ticker_symbol': 'A078930',
 'company': 'GS',
 'report_date': datetime.datetime(2020, 1, 2, 0, 0),
 '발행 전일': datetime.datetime(2019, 12, 30, 0, 0),
 '발행 전전일': nan,
 'report_title': '4분기 실적은 기대수준을 소폭 하회할 전망',
 'opening': 51300,
 'closing': 50000,
 '고가': 51400,
 '전일종가': 51600,
 '전전일종가': nan,
 'recommendation': 'BUY',
 '고가1': 51400.0,
 '고가2': 50700.0,
 '고가3': 51100.0,
 '고가4': 50500.0,
 '고가5': 49250.0,
 '고가6': 49050.0,
 '고가7': 48600.0,
 '고가8': 48150.0,
 '고가9': 48700.0,
 '고가10': 48900.0,
 '고가11': 48350.0,
 '고가12': 48850.0,
 '고가13': 48900.0,
 '고가14': 48700.0,
 '고가15': 48800.0,
 '고가16': 48650.0,
 '고가17': 48050.0,
 '고가18': 46850.0,
 '고가19': 46700.0,
 '고가20': 46750.0,
 '고가21': 45850.0,
 '고가22': 45300.0,
 '고가23': 45250.0,
 '고가24': 47050.0,
 '고가25': 46550.0,
 '고가26': 45900.0,
 '고가27': 46700.0,
 '고가28': 46850.0,
 '고가29': 47100.0,
 '고가30': 46250.0,
 '고가31': 46450.0,
 '고가32': 46150.0,
 '고가33': 46100.0,
 '고가34': 45500.0,
 '고가35': 44650.0,
 '고가36': 43600.0,
 '고가37':

In [30]:
merged_df['author'].value_counts()

   이동헌    342
   박종선    310
김경민       271
   한상원    262
김현용       255
         ... 
유경하         1
정지윤         1
엄태웅         1
김영건         1
   신언범      1
Name: author, Length: 795, dtype: int64